https://adventofcode.com/2022/day/16

In [1]:
import re
from collections import deque, defaultdict
from itertools import combinations

import networkx as nx

In [2]:
with open("data/16.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
"""

In [4]:
def load_data(data):
    D = {}
    for line in data.strip().splitlines():
        fr = int(re.search(r"[+-]?\d+", line).group())
        valves = re.findall(r"[A-Z][A-Z]", line)
        D[valves[0]] = (fr, valves[1:])
    return D

In [5]:
testcave = load_data(testdata)
testcave

{'AA': (0, ['DD', 'II', 'BB']),
 'BB': (13, ['CC', 'AA']),
 'CC': (2, ['DD', 'BB']),
 'DD': (20, ['CC', 'AA', 'EE']),
 'EE': (3, ['FF', 'DD']),
 'FF': (0, ['EE', 'GG']),
 'GG': (0, ['FF', 'HH']),
 'HH': (22, ['GG']),
 'II': (0, ['AA', 'JJ']),
 'JJ': (21, ['II'])}

In [36]:
def open_valves(data, timelimit=30, start="AA"):
    cave = load_data(data)
    flowrates = get_flowrates(cave, start)
    distances = get_distances(cave, flowrates)
    high_score = 0
    high_score_path = None
    q = deque([([start], 0, timelimit)])
    while q:
        pth, score, time_remaining = q.popleft()
        if score > high_score:
            high_score = score
            high_score_path = pth
        for node in extend_path(pth, flowrates, distances, time_remaining, score):
            q.append(node)
    return high_score, high_score_path

def extend_path(pth, flowrates, distances, time_remaining, score):
    for v in set(flowrates).difference(pth):
        dist = distances[frozenset((v, pth[-1]))]
        if dist < time_remaining:
            new_pth = pth + [v]
            new_time_remaining = time_remaining - dist
            new_score = score + new_time_remaining * flowrates[v]
            yield new_pth, new_score, new_time_remaining

def get_flowrates(cave, start="AA"):
    return {valve: flowrate for (valve, (flowrate, neighbors)) in cave.items() if flowrate or valve == start}

def get_distances(cave, flowrates=None):
    if flowrates is None:
        flowrates = get_flowrates(cave)
    g = nx.Graph()
    for (valve, (_, neighbors)) in cave.items():
        for nabe in neighbors:
            g.add_edge(valve, nabe)
    distances = {}
    for a, b in combinations(flowrates, 2):
        distances[frozenset((a, b))] = nx.shortest_path_length(g, a, b) + 1
    return distances

In [37]:
%%time
open_valves(testdata)

CPU times: user 2.56 ms, sys: 180 µs, total: 2.74 ms
Wall time: 2.74 ms


(1651, ['AA', 'DD', 'BB', 'JJ', 'HH', 'EE', 'CC'])

Part 1

In [38]:
%%time
open_valves(data)

CPU times: user 606 ms, sys: 0 ns, total: 606 ms
Wall time: 616 ms


(1559, ['AA', 'KM', 'IC', 'GB', 'OE', 'KT', 'AK'])

Part 2

In [48]:
def open_valves_two_player(data, timelimit=26, start="AA"):
    cave = load_data(data)
    flowrates = get_flowrates(cave, start)
    distances = get_distances(cave, flowrates)
    subset_scores = defaultdict(int)
    q = deque([([start], 0, timelimit)])
    while q:
        pth, score, time_remaining = q.popleft()
        pathset = frozenset(pth)
        subset_scores[pathset] = max(subset_scores[pathset], score)
        for node in extend_path(pth, flowrates, distances, time_remaining, score):
            q.append(node)
    return max(subset_scores[a] + subset_scores[b] for a, b in combinations(subset_scores, 2) if len(a & b) == 1)

In [49]:
%%time
open_valves_two_player(testdata)

CPU times: user 4.75 ms, sys: 39 µs, total: 4.79 ms
Wall time: 4.7 ms


1707

In [50]:
%%time
open_valves_two_player(data)

CPU times: user 962 ms, sys: 0 ns, total: 962 ms
Wall time: 965 ms


2191

Intellectual honesty note: After spending hours trying to make the two player version work with both players running at the same time, I went to the Reddit forum and found the hint that the two players could run separately, then their scores could be combined if their paths did not overlap.